# Example of Tucker Riemaniannian optimization 

In [2]:
# we have to set env variable to use propack, if we want to use sparse2tuck
import os
os.environ["SCIPY_USE_PROPACK"] = "1"
import torch
import numpy as np
import tucker_riemopt
from tucker_riemopt import backend as back
from tucker_riemopt import set_backend
from tucker_riemopt.tucker.tucker import Tucker
import tucker_riemopt.tucker.riemannian as riemann

###### Basic working tensors 
* $\text{Omega_dense} \approx R(0,1) $ is dense mask tensor 
* $\text{Omega_tucker}$ is Tucker representation of mask tensor
* $\text{X_dense} \approx R(0,1)$ is first target tensor approximation (dense tensor)
* $\text{X_tucker}$ is Tucker representation of first approximation

###### Examples of target tensors
* $\text{Sin_dense}(i_1,\ldots,i_d) = sin(i_1 + \ldots + i_d)$ is 2-rank target tensor 
* $\text{Sin_tucker}$ is Tucker representation of Sinus tensor
* $\text{Hilbert_dense}(i_1,\ldots,i_d) = \frac{1}{i_1 + \ldots + i_d + d}$ is target tensor with exponentially decaying singular numbers
* $\text{Hilbert_tucker}$ is Tucker representation of Hilbert tensor

In [5]:
#init tensors and mask
size = 32
set_backend("pytorch")

Sin_dense     = back.zeros([size,size,size])
Hilbert_dense = back.zeros([size,size,size])
Omega_dense   = back.zeros([size,size,size])
X_dense       = back.zeros([size,size,size])

for i in range(size):
    for j in range(size):
        for k in range(size):
            Sin_dense[i][j][k]     = np.sin(i + j + k)
            Hilbert_dense[i][j][k] = 1 / (i + j + k + 3)
            Omega_dense[i][j][k]   = 1 if np.random.uniform() < 0.5 else 0
            X_dense[i][j][k]       = np.random.uniform()

Omega_tucker = Tucker.from_dense(Omega_dense)
Sin_tucker   = Tucker.from_dense(Sin_dense)
X_tucker     = Tucker.from_dense(X_dense)

In [14]:
def Omega_projection(dense_tensor, Omega = Omega_dense):
    #Projection onto known tensor elements, given by dense Omega tensor
    #elementwise product
    return dense_tensor * Omega

def Tucker_Omega_projection(tucker_tensor, Omega = Omega_tucker):
    #Projection tensor given by tucker-format onto known tensor elements, given by dense Omega tensor
    #elementwise product in Tucker format
    return Tucker.from_dense(Omega_projection(tucker_tensor.to_dense()))

def Euclidean_grad(X, Target_tensor = Sin_tucker):
    return Tucker_Omega_projection(Target_tensor) - Tucker_Omega_projection(X)

def f(X, A = Sin_tucker):
    return 1/2 * (Tucker_Omega_projection(A) - Tucker_Omega_projection(X)).norm() ** 2

def line_search(eta, X):
    proj_eta = Tucker_Omega_projection(eta)
    return proj_eta.flat_inner(Euclidean_grad(X)) / (proj_eta.norm() ** 2)

def retraction(X, xi, r):
    return (X + xi).round([r, r, r])
    
# def Riemannian_grad(Grad, X):
#     tucker_riemopt.tucker.riemannian

In [15]:
xi       = riemann.grad(f, X_tucker)
eta      = -xi[0].construct()
alpha    = line_search(eta, X_tucker)
X_tucker = retraction(X_tucker, alpha * eta, 2)

In [16]:
max_iter = 1000

for k in range(max_iter):
    xi    = riemann.grad(f, X_tucker)
    eta   = -xi[0].construct() + riemann.project(X_tucker, eta).construct()
    alpha = line_search(eta, X_tucker)
    X_tucker     = retraction(X_tucker, alpha*eta, 2)
    eps = back.sqrt(2*xi[1]) / Sin_tucker.norm()
    
    if k%50 ==0:
        print(eps)
    if eps <1e-3:
        break

tensor(0.0059, grad_fn=<DivBackward0>)
tensor(0.0052, grad_fn=<DivBackward0>)
tensor(0.0048, grad_fn=<DivBackward0>)


KeyboardInterrupt: 